In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

from statsmodels.api import add_constant
from numpy.linalg import lstsq
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from matplotlib import pyplot as plt

from skglm import GeneralizedLinearEstimator
from skglm.penalties import SCAD

import jax, jax.numpy as jnp
from jaxopt import LBFGS

from numba import njit
import optax

In [2]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
G = 3
GF = np.array((1, 1, 1))

In [4]:
from linearmodels.datasets import wage_panel

wage_panel_df = wage_panel.load()

wage_panel_df.set_index(["nr", "year"], inplace=True)
y = wage_panel_df["lwage"]
x = wage_panel_df.drop(columns=["occupation", "lwage", "black", "hisp", "educ"])
display(x)
display(y)

exper  hours  married  union  expersq
nr    year                                       
13    1980      1   2672        0      0        1
      1981      2   2320        0      1        4
      1982      3   2940        0      0        9
      1983      4   2960        0      0       16
      1984      5   3071        0      0       25
...           ...    ...      ...    ...      ...
12548 1983      8   2080        1      0       64
      1984      9   2080        1      1       81
      1985     10   2080        1      0      100
      1986     11   2080        1      1      121
      1987     12   3380        1      1      144

[4360 rows x 5 columns]

nr     year
13     1980    1.197540
       1981    1.853060
       1982    1.344462
       1983    1.433213
       1984    1.568125
                 ...   
12548  1983    1.591879
       1984    1.212543
       1985    1.765962
       1986    1.745894
       1987    1.466543
Name: lwage, Length: 4360, dtype: float64

In [5]:
N = x.index.get_level_values(0).nunique()
T = x.index.get_level_values(1).nunique()
K = x.shape[1]
N, T, K

(545, 8, 5)

In [6]:
x = x.values.reshape(N, T, K)
y = y.values.reshape(N, T, 1)
x, y

(array([[[   1, 2672,    0,    0,    1],
         [   2, 2320,    0,    1,    4],
         [   3, 2940,    0,    0,    9],
         ...,
         [   6, 2864,    0,    0,   36],
         [   7, 2994,    0,    0,   49],
         [   8, 2640,    0,    0,   64]],
 
        [[   4, 2484,    0,    0,   16],
         [   5, 2804,    0,    0,   25],
         [   6, 2530,    0,    0,   36],
         ...,
         [   9, 2164,    0,    0,   81],
         [  10, 2749,    0,    0,  100],
         [  11, 2476,    0,    0,  121]],
 
        [[   4, 2332,    1,    0,   16],
         [   5, 2116,    1,    0,   25],
         [   6, 2500,    1,    0,   36],
         ...,
         [   9, 2340,    1,    0,   81],
         [  10, 2340,    1,    0,  100],
         [  11, 2340,    1,    0,  121]],
 
        ...,
 
        [[   4, 2008,    1,    0,   16],
         [   5, 3190,    0,    0,   25],
         [   6, 2584,    0,    0,   36],
         ...,
         [   9, 2290,    1,    0,   81],
         [  10, 31

In [7]:
x = x[:30]
y = y[:30]
N = 30
x

array([[[   1, 2672,    0,    0,    1],
        [   2, 2320,    0,    1,    4],
        [   3, 2940,    0,    0,    9],
        ...,
        [   6, 2864,    0,    0,   36],
        [   7, 2994,    0,    0,   49],
        [   8, 2640,    0,    0,   64]],

       [[   4, 2484,    0,    0,   16],
        [   5, 2804,    0,    0,   25],
        [   6, 2530,    0,    0,   36],
        ...,
        [   9, 2164,    0,    0,   81],
        [  10, 2749,    0,    0,  100],
        [  11, 2476,    0,    0,  121]],

       [[   4, 2332,    1,    0,   16],
        [   5, 2116,    1,    0,   25],
        [   6, 2500,    1,    0,   36],
        ...,
        [   9, 2340,    1,    0,   81],
        [  10, 2340,    1,    0,  100],
        [  11, 2340,    1,    0,  121]],

       ...,

       [[   2, 2000,    0,    0,    4],
        [   3, 1906,    0,    0,    9],
        [   4, 1110,    0,    0,   16],
        ...,
        [   7, 2459,    0,    0,   49],
        [   8, 3930,    0,    1,   64],
        [

In [458]:
beta = np.ones((K, N))
mu = np.ones((N, 1))
alpha = np.ones((K, G))
num_parms = K * N + N + K * G
beta.shape, mu.shape, alpha.shape, num_parms

((3, 30), (30, 1), (3, 3), 129)

In [474]:
# @njit
def objective_value(y, x, beta, alpha, mu, kappa):
    base = ((y - np.sum(x * beta.T[:, None, :], axis=2) - mu) ** 2).mean()
    penalty = np.mean(np.prod(np.linalg.norm(beta[:, :, None] - alpha[:, None, :], axis=0), axis=1)) * kappa
    return base + penalty

## NOTE without individual effects
def objective_value_without_individual_effects(y, x, beta, alpha, kappa):
    y = np.squeeze(y, axis=2)
    base = ((y - np.sum(x * beta.T[:, None, :], axis=2)) ** 2).mean()
    penalty = np.mean(np.prod(np.linalg.norm(beta[:, :, None] - alpha[:, None, :], axis=0), axis=1)) * kappa
    return base + penalty

In [ ]:
# NOTE for some reason this is slower than the non-jnp version
@jax.jit
def jnp_objective_value(y, x, beta, alpha, mu, kappa):
    base = ((y - jnp.sum(x * beta.T[:, None, :], axis=2) - mu) ** 2).mean()
    penalty = jnp.mean(jnp.prod(jnp.linalg.norm(beta[:, :, None] - alpha[:, None, :], axis=0), axis=1)) * kappa
    return base + penalty

In [291]:
def unpack(theta):
    beta = theta[:K * N].reshape(K, N)
    mu = theta[K * N:K * N + N].reshape(N, 1)
    alpha = theta[K * N + N:].reshape(K, G)
    return beta, mu, alpha

def pack(beta, mu, alpha):
    return np.concatenate((beta.flatten(), mu.flatten(), alpha.flatten()), axis=0)

In [351]:
# @njit
def obj(theta, kappa=1):
    beta, mu, alpha = unpack(theta)
    return objective_value(y, x, beta, alpha, mu, kappa)

In [ ]:
# Get Initial
# TODO think of better initialization technique
def _generate_initial_estimates(y, x, N, T, K, G):
    beta_init = np.zeros_like(beta)

    for i in range(N):
        beta_init[:, i:i+1] = lstsq(x[i].reshape(T, K), y[i].reshape(T, 1))[0]
    alpha_init = KMeans(n_clusters=G).fit(beta_init.T).cluster_centers_.T

    for j in range(G):
        if (np.abs(beta_init.T - alpha_init[:, j]).min() < 1e-2):
            alpha_init[:, j] += 1e-1 * np.sign(alpha_init[:, j])

    mu_init = np.mean(y, axis=1)

    return beta_init, alpha_init, mu_init

In [304]:
alpha_fixed = np.zeros((K, G))

def unpack_local(theta):
    beta = theta[: K * N].reshape(K, N)
    mu = theta[K * N : K * N + N].reshape(N, 1)
    alpha = alpha_fixed.copy()
    alpha[:, 0 : 1] = theta[K * N + N :].reshape(K, 1)
    return beta, mu, alpha

unpack_local(np.ones(K * N + N + K).reshape(-1))

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [

In [381]:
_generate_initial_estimates(y, x, N, T, K, G)

(array([[ 1.62554507e-01,  1.95135027e-01,  2.98576416e-02,
          4.27584412e-01,  4.88089115e-01,  5.40477178e-01,
          2.98463060e-01,  3.24170055e-01,  3.80320640e-01,
          5.05140190e-01,  3.51968779e-01,  1.55873354e+00,
          2.79857103e-02,  4.88771107e-02,  2.37740751e-01,
          6.06142656e-02, -3.88527825e-01,  1.01349215e+00,
          6.27419161e-01,  1.56768231e-01, -2.42368523e-01,
          1.48416043e-01,  9.66717136e-01,  5.39662321e-02,
         -1.08939439e-01,  1.44806246e-01,  6.83336826e-01,
          1.74249382e-01,  5.17246445e-01,  8.48732133e-01],
        [ 3.60941505e-04,  3.38147311e-04, -3.02041939e-04,
          1.50312545e-04, -7.78647367e-05, -2.75448050e-04,
          4.20395429e-04,  2.38616180e-04, -3.63103562e-05,
         -7.77520986e-05,  3.65944706e-04, -1.25420584e-03,
          6.78013274e-04,  8.95483357e-04,  5.75157952e-04,
          6.83565801e-04, -4.46636150e-04, -4.93480592e-04,
          6.67785958e-05,  5.30258381e-

In [408]:
beta, alpha, mu = _generate_initial_estimates(y, x, N, T, K, G)

obj_value = np.inf
last_obj_value = np.inf
for i in range(1000):
    for j in range(G):
        alpha_fixed = alpha.copy()

        def unpack_local(theta):
            beta = theta[: K * N].reshape(K, N)
            mu = theta[K * N : K * N + N].reshape(N, 1)
            alpha = alpha_fixed.copy()
            alpha[:, j:j+1] = theta[K * N + N :].reshape(K, 1)
            return beta, mu, alpha

        def obj_local(theta, kappa=0.1):
            beta, mu, alpha = unpack_local(theta)
            return objective_value(y, x, beta, alpha, mu, 10)

        def pack_local(beta, mu, alpha):
            return np.concatenate((beta.flatten(), mu.flatten(), alpha[:, j].flatten()), axis=0)

        minimizer = opt.minimize(obj_local, pack_local(beta, mu, alpha), method="Nelder-Mead", options={"maxiter": 1000}, tol=1e-6)
        beta, mu, alpha = unpack_local(minimizer.x)
        obj_value = minimizer.fun
        print(f"Iteration {i}, Group {j}, Objective Value: {obj_value:.6f}")
    if np.abs(obj_value - last_obj_value) < 1e-6:
        print("Convergence reached.")
        break
    last_obj_value = obj_value

beta, mu, alpha

Iteration 0, Group 0, Objective Value: 18.749299
Iteration 0, Group 1, Objective Value: 17.302432
Iteration 0, Group 2, Objective Value: 16.286099
Iteration 1, Group 0, Objective Value: 15.783162
Iteration 1, Group 1, Objective Value: 14.736210
Iteration 1, Group 2, Objective Value: 13.818141
Iteration 2, Group 0, Objective Value: 13.478842
Iteration 2, Group 1, Objective Value: 12.622121
Iteration 2, Group 2, Objective Value: 11.873728
Iteration 3, Group 0, Objective Value: 11.610537
Iteration 3, Group 1, Objective Value: 10.996719
Iteration 3, Group 2, Objective Value: 10.403128
Iteration 4, Group 0, Objective Value: 10.164488
Iteration 4, Group 1, Objective Value: 9.620935
Iteration 4, Group 2, Objective Value: 9.132047
Iteration 5, Group 0, Objective Value: 8.913050
Iteration 5, Group 1, Objective Value: 8.453475
Iteration 5, Group 2, Objective Value: 8.023384
Iteration 6, Group 0, Objective Value: 7.790823
Iteration 6, Group 1, Objective Value: 7.399612
Iteration 6, Group 2, Objec

(array([[ 1.31893769e-01,  1.36544985e-01,  1.43071639e-01,
          1.09559595e-01,  1.54086819e-01,  1.89131840e-01,
          1.36078300e-01,  1.27899547e-01,  2.03580160e-01,
          1.63903237e-01,  1.41526110e-01,  1.74455775e-01,
          1.34488355e-03,  1.40494500e-01,  1.35229251e-01,
          1.44500261e-01, -1.15495406e-02,  1.61186169e-01,
          1.53467908e-01,  1.37415779e-01, -6.31811353e-03,
          1.50610535e-01,  1.73604603e-01,  1.47414296e-01,
         -5.34024717e-03,  1.42944362e-01,  1.53816097e-01,
          1.11839663e-01,  1.61853438e-01,  1.56582264e-01],
        [ 2.19484348e-06,  2.32190589e-05, -7.63305958e-04,
          1.42346641e-05, -9.52000281e-05, -2.33806198e-05,
          5.08693113e-05,  1.23692268e-05, -7.60269942e-05,
         -2.66843756e-04,  2.93100872e-04, -2.13804350e-04,
          6.90127866e-04,  4.56341423e-05,  8.46865077e-06,
          1.31195394e-04, -3.55207771e-04, -6.80014259e-05,
          4.47715262e-06,  4.63812648e-

In [409]:
beta, alpha, mu = _generate_initial_estimates(y, x, N, T, K, G)

obj_value = np.inf
last_obj_value = np.inf
for i in range(1000):
    for j in range(G):
        alpha_fixed = alpha.copy()

        def unpack_local(theta):
            beta = theta[: K * N].reshape(K, N)
            mu = theta[K * N : K * N + N].reshape(N, 1)
            alpha = alpha_fixed.copy()
            alpha[:, j : j + 1] = theta[K * N + N :].reshape(K, 1)
            return beta, mu, alpha

        def obj_local(theta, kappa=0.1):
            beta, mu, alpha = unpack_local(theta)
            return objective_value(y, x, beta, alpha, mu, 10)

        def pack_local(beta, mu, alpha):
            return np.concatenate((beta.flatten(), mu.flatten(), alpha[:, j].flatten()), axis=0)

        minimizer = opt.minimize(
            obj_local, pack_local(beta, mu, alpha), method="Nelder-Mead", options={"maxiter": 1000}, tol=1e-6
        )
        beta, mu, alpha = unpack_local(minimizer.x)
        obj_value = minimizer.fun
        print(f"Iteration {i}, Group {j}, Objective Value: {obj_value:.6f}")
    if np.abs(obj_value - last_obj_value) < 1e-6:
        print("Convergence reached.")
        break
    last_obj_value = obj_value

beta, mu, alpha

Iteration 0, Group 0, Objective Value: 19.418542
Iteration 0, Group 1, Objective Value: 18.220706
Iteration 0, Group 2, Objective Value: 17.027054
Iteration 1, Group 0, Objective Value: 15.931869
Iteration 1, Group 1, Objective Value: 15.517472
Iteration 1, Group 2, Objective Value: 14.466169
Iteration 2, Group 0, Objective Value: 13.597664
Iteration 2, Group 1, Objective Value: 13.296144
Iteration 2, Group 2, Objective Value: 12.515045
Iteration 3, Group 0, Objective Value: 11.787765
Iteration 3, Group 1, Objective Value: 11.501923
Iteration 3, Group 2, Objective Value: 10.874305
Iteration 4, Group 0, Objective Value: 10.203531
Iteration 4, Group 1, Objective Value: 9.925890
Iteration 4, Group 2, Objective Value: 9.449935
Iteration 5, Group 0, Objective Value: 9.007961
Iteration 5, Group 1, Objective Value: 8.772310
Iteration 5, Group 2, Objective Value: 8.330843
Iteration 6, Group 0, Objective Value: 7.912577
Iteration 6, Group 1, Objective Value: 7.683072
Iteration 6, Group 2, Objec

(array([[ 1.35899885e-01,  1.40996431e-01,  1.45713815e-01,
          1.16960605e-01,  1.57532281e-01,  1.91240335e-01,
          1.40587096e-01,  1.32662526e-01,  2.04797125e-01,
          1.65520016e-01,  1.45055859e-01,  1.77357911e-01,
          1.47320192e-03,  1.44131106e-01,  1.39012747e-01,
          1.48226297e-01, -1.04779817e-02,  1.64497748e-01,
          1.56868269e-01,  1.41286826e-01, -6.63335038e-03,
          1.53952057e-01,  1.76449509e-01,  1.50964351e-01,
         -5.83508898e-03,  1.46804987e-01,  1.56956335e-01,
          1.18330402e-01,  1.64686294e-01,  1.59810014e-01],
        [ 2.41173271e-06,  2.76671292e-05, -7.90918757e-04,
          1.72293872e-05, -9.86130214e-05, -2.69140554e-05,
          5.76700939e-05,  1.42559664e-05, -8.02518130e-05,
         -2.71206949e-04,  2.94388037e-04, -2.16415902e-04,
          6.84740946e-04,  5.16393669e-05,  9.81441649e-06,
          1.33897692e-04, -3.56367710e-04, -7.13506345e-05,
          5.52160964e-06,  4.64418459e-

In [442]:
import cma

In [444]:
beta, alpha, _ = _generate_initial_estimates(y, x, N, T, K, G)

obj_value = np.inf
last_obj_value = np.inf
init_BFGS = True
for i in range(1000):
    for j in range(G):
        alpha_fixed = alpha.copy()

        def unpack_local(theta):
            beta = theta[: K * N].reshape(K, N)
            # mu = theta[K * N : K * N + N].reshape(N, 1)
            alpha = alpha_fixed.copy()
            alpha[:, j : j + 1] = theta[K * N :].reshape(K, 1)
            return beta, alpha

        def obj_local(theta, kappa=0.1):
            beta, alpha = unpack_local(theta)
            return objective_value_without_individual_effects(y, x, beta, alpha, 10)

        def pack_local(beta, alpha):
            return np.concatenate((beta.flatten(), alpha[:, j].flatten()), axis=0)

        if init_BFGS:
            minimizer = opt.minimize(
                obj_local, pack_local(beta, alpha), method="BFGS", options={"maxiter": 1000}, tol=1e-6
            )
            beta, alpha = unpack_local(minimizer.x)
            obj_value = minimizer.fun
            print(f"BFGS Iteration {i}, Group {j}, Objective Value: {obj_value:.6f}")
        else:
            minimizer = opt.basinhopping(
                obj_local, pack_local(beta, alpha), minimizer_kwargs={"method": "BFGS"}
            )
            beta, alpha = unpack_local(minimizer.x)
            obj_value = minimizer.fun
            print(f"Nelder-Mead Iteration {i}, Group {j}, Objective Value: {obj_value:.6f}")
    if np.abs(obj_value - last_obj_value) < 1e-6:
        print("Convergence reached.")
        if init_BFGS:
            init_BFGS = False
        else:
            break
    last_obj_value = obj_value

beta, alpha

BFGS Iteration 0, Group 0, Objective Value: 0.122353
BFGS Iteration 0, Group 1, Objective Value: 0.079443
BFGS Iteration 0, Group 2, Objective Value: 0.072277
BFGS Iteration 1, Group 0, Objective Value: 0.070472
BFGS Iteration 1, Group 1, Objective Value: 0.070377
BFGS Iteration 1, Group 2, Objective Value: 0.070377
BFGS Iteration 2, Group 0, Objective Value: 0.070342
BFGS Iteration 2, Group 1, Objective Value: 0.070328
BFGS Iteration 2, Group 2, Objective Value: 0.070328
BFGS Iteration 3, Group 0, Objective Value: 0.070328
BFGS Iteration 3, Group 1, Objective Value: 0.070328
BFGS Iteration 3, Group 2, Objective Value: 0.070328
Convergence reached.
Nelder-Mead Iteration 4, Group 0, Objective Value: 0.070328
Nelder-Mead Iteration 4, Group 1, Objective Value: 0.070328
Nelder-Mead Iteration 4, Group 2, Objective Value: 0.070328
Convergence reached.


(array([[ 2.34573412e-01,  2.33155870e-01,  2.34035669e-01,
          2.32787796e-01,  5.39209472e-01,  5.38797561e-01,
          2.33332123e-01,  3.40256563e-01,  3.48431087e-01,
          3.43792085e-01,  3.43228494e-01,  5.39816886e-01,
          2.32985198e-01,  2.31653790e-01,  2.34329863e-01,
          2.33152959e-01,  5.39359602e-01,  5.40059655e-01,
          5.39898453e-01,  2.32204624e-01,  2.32482567e-01,
          2.33146885e-01,  5.39899378e-01,  2.33188833e-01,
          2.33304755e-01,  2.33153975e-01,  5.39823317e-01,
          3.35766114e-01,  5.35223841e-01,  3.43549307e-01],
        [ 3.57026920e-04,  2.90855111e-04,  2.68999989e-04,
          4.61835641e-04, -1.24127352e-04, -2.51888443e-04,
          4.81272738e-04,  2.13723963e-04, -6.34546215e-05,
         -5.70798064e-05,  3.52008102e-04,  2.61975084e-04,
          4.04810285e-04,  6.45768202e-04,  5.61296413e-04,
          3.88753941e-04,  3.24748273e-05,  1.56263557e-04,
          1.54366315e-04,  4.20525193e-

In [484]:
import pickle

example = pickle.load(open("dgp3-example-3.pkl", "rb"))

In [486]:
x_example = example[0]
y_example = np.reshape(example[1], (300, 100, 1))
g_example = example[2]
alpha_example = example[3]
beta_example = example[4]
beta_example

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [487]:
y_example.shape

(300, 100, 1)

In [488]:
x = x_example
y = y_example
N, T, K = x.shape
G = 3

In [491]:
def objective_value_without_individual_effects(y, x, beta, alpha, kappa):
    y = np.squeeze(y, axis=2)
    base = ((y - np.sum(x * beta.T[:, None, :], axis=2)) ** 2).mean()
    penalty = np.mean(np.prod(np.linalg.norm(beta[:, :, None] - alpha[:, None, :], axis=0), axis=1)) * kappa
    return base + penalty

In [495]:
def _generate_initial_estimates(y, x, N, T, K, G):
    beta = np.zeros((K, N))
    beta_init = np.zeros_like(beta)

    for i in range(N):
        beta_init[:, i : i + 1] = lstsq(x[i].reshape(T, K), y[i].reshape(T, 1))[0]
    alpha_init = KMeans(n_clusters=G).fit(beta_init.T).cluster_centers_.T

    for j in range(G):
        if np.abs(beta_init.T - alpha_init[:, j]).min() < 1e-2:
            alpha_init[:, j] += 1e-1 * np.sign(alpha_init[:, j])

    mu_init = np.mean(y, axis=1)

    return beta_init, alpha_init, mu_init

In [498]:
beta, alpha, _ = _generate_initial_estimates(y, x, N, T, K, G)

obj_value = np.inf
last_obj_value = np.inf
for i in range(1000):
    for j in range(G):
        alpha_fixed = alpha.copy()

        def unpack_local(theta):
            beta = theta[: K * N].reshape(K, N)
            # mu = theta[K * N : K * N + N].reshape(N, 1)
            alpha = alpha_fixed.copy()
            alpha[:, j : j + 1] = theta[K * N :].reshape(K, 1)
            return beta, alpha

        def obj_local(theta, kappa=0.1):
            beta, alpha = unpack_local(theta)
            return objective_value_without_individual_effects(y, x, beta, alpha, 10)

        def pack_local(beta, alpha):
            return np.concatenate((beta.flatten(), alpha[:, j].flatten()), axis=0)

        minimizer = opt.minimize(
            obj_local, pack_local(beta, alpha), method="L-BFGS-B", options={"maxiter": 1000}, tol=1e-6
        )
        beta, alpha = unpack_local(minimizer.x)
        obj_value = minimizer.fun
        print(f"BFGS Iteration {i}, Group {j}, Objective Value: {obj_value:.6f}")
        # else:
        #     minimizer = opt.minimize(
        #         obj_local, pack_local(beta, alpha), method="Nelder-Mead", options={"adaptive": True}, tol=1e-6
        #     )
        #     beta, alpha = unpack_local(minimizer.x)
        #     obj_value = minimizer.fun
        #     print(f"Nelder-Mead Iteration {i}, Group {j}, Objective Value: {obj_value:.6f}")
    if np.abs(obj_value - last_obj_value) < 1e-6:
        print("Convergence reached.")
        break
    last_obj_value = obj_value

beta, alpha

BFGS Iteration 0, Group 0, Objective Value: 2.685249
BFGS Iteration 0, Group 1, Objective Value: 2.153861
BFGS Iteration 0, Group 2, Objective Value: 1.498722
BFGS Iteration 1, Group 0, Objective Value: 1.464179
BFGS Iteration 1, Group 1, Objective Value: 1.455412
BFGS Iteration 1, Group 2, Objective Value: 1.428024
BFGS Iteration 2, Group 0, Objective Value: 1.424138
BFGS Iteration 2, Group 1, Objective Value: 1.422813
BFGS Iteration 2, Group 2, Objective Value: 1.359962
BFGS Iteration 3, Group 0, Objective Value: 1.334502
BFGS Iteration 3, Group 1, Objective Value: 1.328425
BFGS Iteration 3, Group 2, Objective Value: 1.286920
BFGS Iteration 4, Group 0, Objective Value: 1.275534
BFGS Iteration 4, Group 1, Objective Value: 1.270954
BFGS Iteration 4, Group 2, Objective Value: 1.242763
BFGS Iteration 5, Group 0, Objective Value: 1.239158
BFGS Iteration 5, Group 1, Objective Value: 1.234348
BFGS Iteration 5, Group 2, Objective Value: 1.223503
BFGS Iteration 6, Group 0, Objective Value: 1.

(array([[1.08254596, 2.08868309, 1.08254598, 1.08254596, 1.08194338,
         2.0888302 , 2.97507425, 2.08570707, 2.9750743 , 2.08850451,
         1.08254599, 2.97507433, 2.97507432, 2.9750743 , 1.08254594,
         1.08254598, 2.97507433, 2.97507425, 1.082546  , 1.08254815,
         2.9750743 , 1.08254599, 2.9750743 , 2.05028778, 2.04179113,
         2.97507429, 1.08254596, 2.0886856 , 1.08254594, 1.08254598,
         1.08254601, 2.08868348, 2.97507432, 1.08254601, 2.97507596,
         2.97507432, 2.97507429, 2.9750743 , 2.08881471, 2.08806719,
         1.08254476, 1.08254594, 2.08640358, 1.08254599, 2.97507431,
         1.08254594, 2.08926972, 2.97507345, 2.08853571, 2.97507433,
         2.97507428, 2.08868344, 2.08868356, 1.08254666, 2.97507431,
         1.08254595, 2.97507431, 2.97507432, 2.97507431, 2.97507429,
         2.97507433, 2.08891709, 2.01817448, 1.08254598, 2.07368978,
         2.97507429, 2.08868354, 1.08254595, 2.97507429, 2.08868454,
         2.05501937, 2.08868347, 2